In [1]:
import argparse

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

try:
    from apex import amp

except ImportError:
    amp = None

# from dataset import LMDBDataset
from pixelsnail import PixelSNAIL
from scheduler import CycleScheduler

import argparse
import sys
import os
from skimage import io

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, utils
import matplotlib.patches as mpatches


# from scheduler import CycleScheduler
from pt_utils import  Embeddings, Trainer, VQVAE, data_sampler, Vqvae2Adaptive
from torch.utils import data
from torch import distributed as dist

from umap import UMAP
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from collections import Counter
from skimage import transform, metrics
import skimage
import pandas as pd
import numpy as np
import glob

from torchsummary import summary

from tqdm.notebook import trange, tqdm
import torchvision
from PIL import Image

import numpy as np
from sklearn.decomposition import PCA
from scipy.ndimage.filters import gaussian_filter
import scipy as sp
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import joblib
import pickle

seed = 51
np.random.seed(seed)
torch.manual_seed(seed)

C:\Users\Администратор\AppData\Local\Temp\ipykernel_2144\339495344.py:55: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


# Create embeddings dataset

In [3]:
# dataset_path = '../data/dataset_512/'
# dataset_path = '../datasets/bc_right_sub_left_minmax_4x_360'
# dataset_path = '../datasets/bc_left_4x_360'
dataset_path = '../datasets/original/o_bc_left_9x_512_360'
# dataset_path = '../datasets/original/o_bc_left_4x_768'

new_dataset_path='../datasets/original/emb_dim_1_n_embed_8192_bc_left_9x_512_360'
if os.path.exists(new_dataset_path) is False:
    os.mkdir(new_dataset_path)

device='cuda'

resize_shape = (512, 512)
# resize_shape = (1024, 1024)

batch_size = 4

transform = transforms.Compose(
    [
        # transforms.Resize(resize_shape),
        # transforms.CenterCrop(resize_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)
model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_9x_512_360/vqvae_002_train_0.01976_test_0.01984.pt'

model =    VQVAE(in_channel=3,
                   channel=128,
                   n_res_block=6,
                   n_res_channel=32,
                   embed_dim=1,
                   n_embed=8192,
                   decay=0.99).to(device)

dataset = datasets.ImageFolder(dataset_path, transform=transform)
device='cuda'

images_embs_t = []
images_embs_b = []

dataset_path = dataset.__dict__['root']
classes_folders = os.listdir(dataset_path)
classes_folders_images = [os.listdir(dataset_path + '/' + folder) for folder in classes_folders]
classes_folders_images_num = [len(os.listdir(dataset_path + '/' + folder)) for folder in classes_folders]

img_transform = dataset.__dict__['transform']

In [8]:
classes_folders_images[i][j][:-5]

'Ultra_Co11-23_part_6_angle_90'

In [4]:
os.mkdir(new_dataset_path + '/top/' )
os.mkdir(new_dataset_path + '/bottom/')

In [5]:
for i in range(len(classes_folders)):
    print(f'Number of folders {i + 1}/{len(classes_folders)}')
    
    os.mkdir(new_dataset_path + '/top/' + classes_folders[i])
    os.mkdir(new_dataset_path + '/bottom/' + classes_folders[i])
        
    for j in tqdm(range(classes_folders_images_num[i]), desc=f'Folder {classes_folders[i]}'):
        image_path = dataset_path + '/' + classes_folders[i] + '/' + classes_folders_images[i][j]

        image = Image.open(image_path)
        image = image.convert("RGB")
        image = img_transform(image)
        image = image.unsqueeze(0).to(device)
        

        model.zero_grad()

        quant_t, quant_b, _, _, _ = model.encode(image)
        
        # quant_t.requires_grad=False
        # quant_b.requires_grad=False

        torch.save(quant_t,new_dataset_path+ '/top/' + classes_folders[i]+'/'+classes_folders_images[i][j][:-5]+'.pt',)
        torch.save(quant_b,new_dataset_path+ '/bottom/' + classes_folders[i]+'/'+classes_folders_images[i][j][:-5]+'.pt')

Number of folders 1/5


Folder Ultra_Co11:   0%|          | 0/3240 [00:00<?, ?it/s]

Number of folders 2/5


Folder Ultra_Co15:   0%|          | 0/3240 [00:00<?, ?it/s]

Number of folders 3/5


Folder Ultra_Co25:   0%|          | 0/3240 [00:00<?, ?it/s]

Number of folders 4/5


Folder Ultra_Co6_2:   0%|          | 0/3240 [00:00<?, ?it/s]

Number of folders 5/5


Folder Ultra_Co8:   0%|          | 0/3240 [00:00<?, ?it/s]

In [ ]:
import argparse
import pickle

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import lmdb
from tqdm import tqdm

from dataset import ImageFileDataset, CodeRow
from vqvae import VQVAE


def extract(lmdb_env, loader, model, device):
    index = 0

    with lmdb_env.begin(write=True) as txn:
        pbar = tqdm(loader)

        for img, _, filename in pbar:
            img = img.to(device)

            _, _, _, id_t, id_b = model.encode(img)
            id_t = id_t.detach().cpu().numpy()
            id_b = id_b.detach().cpu().numpy()

            for file, top, bottom in zip(filename, id_t, id_b):
                row = CodeRow(top=top, bottom=bottom, filename=file)
                txn.put(str(index).encode('utf-8'), pickle.dumps(row))
                index += 1
                pbar.set_description(f'inserted: {index}')

        txn.put('length'.encode('utf-8'), str(index).encode('utf-8'))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--size', type=int, default=256)
    parser.add_argument('--ckpt', type=str)
    parser.add_argument('--name', type=str)
    parser.add_argument('path', type=str)

    args = parser.parse_args()

    device = 'cuda'

    transform = transforms.Compose(
        [
            transforms.Resize(args.size),
            transforms.CenterCrop(args.size),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ]
    )

    dataset = ImageFileDataset(args.path, transform=transform)
    loader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=4)

    model = VQVAE()
    model.load_state_dict(torch.load(args.ckpt))
    model = model.to(device)
    model.eval()

    map_size = 100 * 1024 * 1024 * 1024

    env = lmdb.open(args.name, map_size=map_size)

    extract(env, loader, model, device)

# Train pixelsnail

In [2]:
# CodeRow = namedtuple('CodeRow', ['top', 'bottom', 'filename'])


# class ImageFileDataset(datasets.ImageFolder):
#     def __getitem__(self, index):
#         sample, target = super().__getitem__(index)
#         path, _ = self.samples[index]
#         dirs, filename = os.path.split(path)
#         _, class_name = os.path.split(dirs)
#         filename = os.path.join(class_name, filename)

#         return sample, target, filename


class LMDBDataset(datasets.DatasetFolder):
#     def __init__(self, path):
#         self.env = lmdb.open(
#             path,
#             max_readers=32,
#             readonly=True,
#             lock=False,
#             readahead=False,
#             meminit=False,
#         )


#         self.length = int(txn.get('length'.encode('utf-8')).decode('utf-8'))

#     def __len__(self):
#         return self.length

    def __getitem__(self, index):
        with self.env.begin(write=False) as txn:
            key = str(index).encode('utf-8')

            row = pickle.loads(txn.get(key))

        return torch.from_numpy(row.top), torch.from_numpy(row.bottom), row.filename
    
    
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root: str, folder: str, klass: int, extension: str = "png"):
        self._data = pathlib.Path(root) / folder
        self.klass = klass
        self.extension = extension
        # Only calculate once how many files are in this folder
        # Could be passed as argument if you precalculate it somehow
        # e.g. ls | wc -l on Linux
        self._length = sum(1 for entry in os.listdir(self._data))

    def __len__(self):
        # No need to recalculate this value every time
        return self._length

    def __getitem__(self, index):
        # images always follow [0, n-1], so you access them directly
        return Image.open(self._data / "{}.{}".format(str(index), self.extension))

In [23]:
def train(epoch, loader, model, optimizer, scheduler, device):
    loader = tqdm(loader)

    criterion = nn.CrossEntropyLoss()

    for i, data in tqdm(enumerate(loader)):
        model.zero_grad()

        top = top.to(device)

        if args.hier == 'top':
            target = data
            out, _ = model(data)

        elif args.hier == 'bottom':
            bottom = data.to(device)
            target = bottom
            out, _ = model(data, condition=top)

        loss = criterion(out, target)
        loss.backward()

        if scheduler is not None:
            scheduler.step()
        optimizer.step()

        _, pred = out.max(1)
        correct = (pred == target).float()
        accuracy = correct.sum() / target.numel()

        lr = optimizer.param_groups[0]['lr']
        
        print(f'Train epoch: {epoch + 1}; loss: {loss.item():.5f}; acc: {accuracy:.5f}; lr: {lr:.5f}')

        
def evaluate( epoch, loader, model, optimizer, scheduler, device):
    loader = tqdm(loader)

    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for i, data in tqdm(enumerate(loader)):


            top = top.to(device)

            if args.hier == 'top':
                target = data
                out, _ = model(data)

            elif args.hier == 'bottom':
                bottom = data.to(device)
                target = bottom
                out, _ = model(data, condition=top)

            loss = criterion(out, target)

            _, pred = out.max(1)
            correct = (pred == target).float()
            accuracy = correct.sum() / target.numel()

            lr = optimizer.param_groups[0]['lr']

            print(f'Test epoch: {epoch + 1}; loss: {loss.item():.5f}; acc: {accuracy:.5f}; lr: {lr:.5f}')

            return round(loss.item()), round(accuracy,3)


In [15]:
file_name='Ultra_Co6_2/Ultra_Co6_2-001_part_2_angle_270.pt'
torch.load(dataset_path+'/'+file_name)

tensor([[[[-0.0748, -0.0769, -0.0748,  ..., -0.0769, -0.0637, -0.0996],
          [-0.0912, -0.0969, -0.0934,  ..., -0.0946, -0.0922, -0.1024],
          [-0.0851, -0.0911, -0.0877,  ..., -0.0889, -0.0875, -0.1064],
          ...,
          [-0.0859, -0.0932, -0.0896,  ..., -0.0913, -0.0873, -0.1062],
          [-0.0806, -0.0932, -0.0896,  ..., -0.0899, -0.0902, -0.1067],
          [-0.0637, -0.0851, -0.0822,  ..., -0.0806, -0.0888, -0.0947]]]],
       device='cuda:0', requires_grad=True)

In [22]:
dataset_path = '../datasets/original/emb_dim_1_n_embed_8192_bc_left_9x_512_360/top'

n_gpu = 1
batch_size = 4
val_split = 0.15

dataset = datasets.DatasetFolder(dataset_path, loader=torch.load, extensions=['.pt'])

train_dataset_len = int(len(dataset) * (1 - val_split))
test_dataset_len = len(dataset) - train_dataset_len

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_dataset_len, test_dataset_len],
                                                            generator=torch.Generator().manual_seed(seed))

train_sampler = data_sampler(train_dataset, shuffle=True, distributed=False)
test_sampler = data_sampler(test_dataset, shuffle=True, distributed=False)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size // n_gpu, sampler=train_sampler
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size // n_gpu, sampler=test_sampler
)


In [26]:
batch=32
hier='top'
lr=3e-4
channel=256
n_res_block=4
n_res_channel=256
n_out_res_block=0
n_cond_res_block=3
dropout=0.1
amp='O0'
sched=None
ckpt=None
path=None

device = 'cuda'


if hier == 'top':
    model = PixelSNAIL(
        [32, 32],
        512,
        channel,
        5,
        4,
        n_res_block,
        n_res_channel,
        dropout=dropout,
        n_out_res_block=n_out_res_block,
    )

elif hier == 'bottom':
    model = PixelSNAIL(
        [64, 64],
        512,
        channel,
        5,
        4,
        n_res_block,
        n_res_channel,
        attention=False,
        dropout=dropout,
        n_cond_res_block=n_cond_res_block,
        cond_res_channel=n_res_channel,
    )


# if 'model' in ckpt:
#     model.load_state_dict(torch.load(ckpt))

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

scheduler = CycleScheduler(
    optimizer, lr, n_iter=len(train_loader) * epoch, momentum=None
)

C:\anaconda3\envs\torch\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [27]:
epoch=420
device='cuda'

for i in range(epoch):
    train( i, train_loader, model, optimizer, scheduler, device)
    loss, acc = evaluate( i, test_loader, model, optimizer, scheduler, device)
    torch.save(model.state_dict(), f'checkpoint/pixelsnail_{hier}_{str(i + 1)}_loss_{loss}_acc_{acc}.pt')
         

  0%|          | 0/3443 [00:00<?, ?it/s]

0it [00:00, ?it/s]

ValueError: not enough values to unpack (expected 3, got 2)